In [1]:
import pytesseract
from pdf2image import convert_from_path
from PIL import Image
import cv2
import numpy as np
import os
import json

In [ ]:
# Percorsi input/output
PDF_PATH = "file.pfd"
OUTPUT_JSON = "output_spese.json"

def preprocess_image(image: Image.Image) -> Image.Image:
    # Converti PIL Image in OpenCV image per miglioramento
    cv_image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    gray = cv2.cvtColor(cv_image, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)
    return Image.fromarray(thresh)

def extract_text_from_pdf(pdf_path: str):
    print(f"Converto il PDF in immagini: {pdf_path}")
    images = convert_from_path(pdf_path)
    full_text = ""

    for i, image in enumerate(images):
        print(f"Processo pagina {i + 1}")
        # Preprocessing base (facoltativo)
        processed_image = preprocess_image(image)

        text = pytesseract.image_to_string(processed_image, lang="ita+eng")
        full_text += f"--- Pagina {i + 1} ---\n{text}\n"

    return full_text

def parse_transactions(raw_text: str):
    # Funzione di parsing *molto basilare* da personalizzare secondo il formato dei tuoi estratti conto
    transactions = []
    lines = raw_text.split("\n")
    for line in lines:
        if any(char.isdigit() for char in line) and "€" in line:
            transactions.append({"riga": line.strip()})
    return transactions


In [8]:
#def main():
    #raw_text = extract_text_from_pdf(PDF_PATH)
    #transactions = parse_transactions(raw_text)

    #print(f"Scrivo {len(transactions)} transazioni in {OUTPUT_JSON}")
    #with open(OUTPUT_JSON, "w", encoding="utf-8") as f:
    #    json.dump(transactions, f, indent=2, ensure_ascii=False)

In [5]:
import pdfplumber

In [6]:
# Apri il file PDF
with pdfplumber.open(PDF_PATH) as pdf:
    # Itera su ogni pagina
    for pagina in pdf.pages:
        # Estrai il testo dalla pagina
        testo = pagina.extract_text()
        print("Testo pagina:\n", testo)
        
        # Prova a estrarre tabelle dalla pagina
        tabelle = pagina.extract_tables()
        for i, tabella in enumerate(tabelle):
            print(f"\nTabella {i+1}:")
            for riga in tabella:
                print(riga)


Testo pagina:
 Informativa Riepilogativa Servizi di
pagamento dal 01/06/2025 al 30/06/2025
RINALDI LUCA
VIA MARINO MARMIROLI 1
42020 ALBINEA RE
Imposta di bollo assolta in modo virtuale, ove prevista.
Autorizzazione intendenza di Finanza di Reggio Emilia n. 4369
del 28.03.1983
PER INFORMAZIONI PUO' CONTATTARE IL SUO CONSULENTE DEBORAH TURSI
Dettaglio
DATA OPERAZIONE DATA VALUTA DESCRIZIONE OPERAZIONE IMPORTO A DEBITO IMPORTO A CREDITO
PRELIEVO BANCOMAT
03/06/2025 29/05/2025 50,00
29/05/25,ORE 22:52,TESS.802574245 ALBINEA
BANCOMAT 29/05/25,ORE 23:05,TESS.802574245
03/06/2025 29/05/2025 10,00
MIGUELITA S.R.L
BANCOMAT 29/05/25,ORE 08:56,TESS.802574245
03/06/2025 29/05/2025 1,50
PIRRU LE REGGIANE
BANCOMAT 30/05/25,ORE 21:43,TESS.802574245
04/06/2025 30/05/2025 13,50
DADA BAR E OSTERIA SNC
BANCOMAT 01/06/25,ORE 19:18,TESS.802574245
04/06/2025 01/06/2025 7,23
SIGMA SAN POLO
BONIFICO ISTANTANEO A CREDITO
06/06/2025 06/06/2025 30,00
CATTANI ALESSIO Rif. 1191251576402413 Regalo
BANCOMAT 04/06/2